In [2]:
%reload_ext autoreload
%autoreload 2
from datasets import load_dataset


#mc4の読み込み
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)

"""
#oscarなども読み込める
#ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)

"""

dataset=mc4_dataset

/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for mc4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mc4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
from src.cleaner.auto_cleaner import clean_text
from src.classifier.DatasetAnnotator import DatasetAnnotator

In [4]:
annotator=DatasetAnnotator(dataset,clean_func=clean_text)

[Errno 2] No such file or directory: 'annotations/good.txt'


In [9]:
import random
#random annotations
max_id=10000

"""
do annotations
Enter: annnotate as "bad"
something: annotate as "good"
q: quit
"""
for i in range(300):
    id=random.randint(0,max_id)
    r=annotator.annotate(id)
    if not r:
        break

In [7]:
annotator.get_text_by_id(1283)

'まさる0798さん\nキッチンで使っています!\n組み立てだけど、グラグラもしずしっかりしています。\n小さくて丁度いい。\nなかなか小さいデスクが見つからなかったので、買って良かったです。\n組み立ては一時間、電動ドリルを使いました。\nサイズはとても小さいので便利です。塗装はナチュラルなので天板はざらつきがあります。\n色が実際にはかなり濃く感じました。白が廃盤じゃなければよかったのですが、色はほぼこげ茶です。\n作るのが大変過ぎました...。\nドライバーは持っていましたがネジを入れようと思っても硬すぎて硬すぎて...。\nなんとかネジを回し切りましたが、手にマメができかけました。\n【お店へのアドバイス】\nよく付属品である先が六角の回すやつ(へ←こんな形のやつ)の先がプラスドライバーのやつを付属した方が良いと思います。\n木なのでかなり固く、回すのが大変という点は前もって通知しておいた方がお客さんの為になるかと思います。\n完成した物については全く不満はありません。\nしっかりとした造りで色合いも良く気に入っています。\n【購入を検討している方、購入して組立がまだの方へ】\n足を下に入れることを考えると、下の台は上の方の穴で固定した方が良いと思います。\n上で固定してもスペースはかごなどを入れない限り全然あります。\nまた、上記の通り一度固定すると変更は容易ではないので気を付けてください。\n昔のテレビ台とか本棚みたいに、突起の上に乗せる形で下の台の位置を変えれたらいいんですけどね。'